In [1]:
import asyncio
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

## Chat completion

In [2]:
# chat_completion = OpenAIChatCompletion(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     ai_model_id="o4-mini"
# )

# .env file should contain:
# AZURE_OPENAI_DEPLOYMENT_NAME=o4-mini
# AZURE_OPENAI_ENDPOINT=https://lseg-foundry-demo.cognitiveservices.azure.com/openai/deployments/o4-mini/chat/completions?api-version=2025-01-01-preview
# AZURE_OPENAI_API_KEY=3JzpWqvXZzC7Nohnw8ujaxKozC6KRIBe9Cx89MKFLDevbWDRFZ1cJQQJ99BFACfhMk5XJ3w3AAAAACOGSzYV
# AZURE_OPENAI_API_VERSION=2025-01-01-preview
# AZURE_GPT_4_1_DEPLOYMENT_NAME=gpt-4.1
# AZURE_GPT_4_1_ENDPOINT=https://lseg-foundry-demo.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview
# AZURE_GPT_4_1_API_KEY=3JzpWqvXZzC7Nohnw8ujaxKozC6KRIBe9Cx89MKFLDevbWDRFZ1cJQQJ99BFACfhMk5XJ3w3AAAAACOGSzYV


chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_GPT_4_1_API_KEY"),
    endpoint=os.getenv("AZURE_GPT_4_1_ENDPOINT"),
    deployment_name=os.getenv("AZURE_GPT_4_1_DEPLOYMENT_NAME"),
)

In [3]:
execution_settings = AzureChatPromptExecutionSettings() # Set temperature, max_tokens, etc. as needed

chat_history = ChatHistory()
chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(chat_history=chat_history, settings=execution_settings)

In [4]:
response.content

"Hello! I'm just a program, so I don't have feelings, but I'm ready to help you. How can I assist you today?"

In [29]:
# Add system message
chat_history = ChatHistory()

chat_history.add_message(
    message=ChatMessageContent(
        role=AuthorRole.SYSTEM,
        content="You are a helpful assistant that always answers in rhymes.",
    )
)

chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.USER,
        name="Ani",
        items=[
            TextContent(text="What is the capital of this country?"),
            ImageContent.from_file(file_path="france.png", mime_type="image/png")
        ]
    )
)

response = await chat_completion.get_chat_message_content(chat_history=chat_history, settings=execution_settings)

print(response.content)

In the land of the tricolour waving so free,  
Paris is the capital where all eyes shall be.


## Agent

In [30]:
agent = ChatCompletionAgent(
        service=chat_completion,
        name="Assistant",
        instructions="Answer questions about the world in one sentence.",
    )


In [31]:
response = await agent.get_response(messages="What is the capital of France?")  # Example usage

In [32]:
response.message.content # Convert the response to a dictionary for easier inspection

'Paris is the capital of France.'

In [33]:
## Introduce threads
thread: ChatHistoryAgentThread = None

response = await agent.get_response(messages="What is the capital of France??", thread=thread)
print(response.message.content)

thread = response.thread  # Get the thread from the response
response = await agent.get_response(messages="What was the last question I asked you?", thread=thread)  # Example usage
print(response.message.content)  


Paris is the capital of France.
Your last question was “What is the capital of France?”


In [5]:
## Now introduce plugins and function calling
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"
    
agent = ChatCompletionAgent(
        service=chat_completion,
        name="Host",
        instructions="Answer questions about the menu.",
        plugins=[MenuPlugin()]  # Register the MenuPlugin with the agent
    )

USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    # "What does that cost?",
    # "Thank you",
]

thread: ChatHistoryAgentThread = None

for user_input in USER_INPUTS:
    print(f"# User: {user_input}")
    # 4. Invoke the agent for a response
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"# {response.name}: {response} ")
    thread = response.thread



# User: Hello
# Host: Hello! How can I help you with the menu today? 
# User: What is the special soup?
# Host: The special soup today is Clam Chowder. Would you like to know more about it or see other specials? 


In [37]:
async for message in thread.get_messages():  # Get all messages in the thread
    print(message.to_dict())  # Print each message as a dictionary

await thread.delete() if thread else None


{'role': 'user', 'content': 'Hello'}
{'role': 'assistant', 'content': 'Hello! How can I assist you with our menu today?', 'name': 'Host'}
{'role': 'user', 'content': 'What is the special soup?'}
{'role': 'assistant', 'tool_calls': [{'id': 'call_g19Bbo0fQpk8r1sN9C50kUAI', 'type': 'function', 'function': {'name': 'MenuPlugin-get_specials', 'arguments': '{}'}}], 'name': 'Host'}
{'role': 'tool', 'content': '\n        Special Soup: Clam Chowder\n        Special Salad: Cobb Salad\n        Special Drink: Chai Tea\n        ', 'tool_call_id': 'call_g19Bbo0fQpk8r1sN9C50kUAI'}
{'role': 'assistant', 'content': 'The special soup today is Clam Chowder.', 'name': 'Host'}


In [52]:
## Intermediate steps for tools. Using agent.invoke() instead of agent.get_response() to handle intermediate steps.
# get_respons() is for getting the final answer, blocking until it's ready.
# invoke() is for getting the answer in pieces, allowing you to see the process unfold in real-time. 
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent


async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"# Function Result:> {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"# Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"# {message.name}: {message} ")

     
agent = ChatCompletionAgent(
        service=chat_completion,
        name="Host",
        instructions="Answer questions about the menu.",
        plugins=[MenuPlugin()]  # Register the MenuPlugin with the agent
    )


thread: ChatHistoryAgentThread = None

TASK = "What is the special soup? What does that cost? Thank you"

print(f"# User: '{TASK}'")
async for response in agent.invoke(
    messages=TASK,
    on_intermediate_message=handle_intermediate_steps,
):
    print(f"# {response.name}: {response} ")


# User: 'What is the special soup? What does that cost? Thank you'
# Function Call:> MenuPlugin-get_specials with arguments: {}
# Function Result:> 
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        
# Function Call:> MenuPlugin-get_item_price with arguments: {"menu_item":"Clam Chowder"}
# Function Result:> $9.99
# Host: The special soup is Clam Chowder, and it costs $9.99. 


In [7]:
## Show reasoning model - https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/reasoning?tabs=python-secure%2Cpy


chat_completion_o4_minit = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    # instruction_role for reasoning models
    service_id="o4-mini-reasoning",
    instruction_role="developer",
)

settings = AzureChatPromptExecutionSettings(service_id="o4-mini-reasoning", reasoning_effort="high")  # Set reasoning effort to high for more complex reasoning

agent = ChatCompletionAgent(
    name="ReasoningAgent",
    instructions="""
    You are a helpful assistant.
    """,
    # kernel=kernel,
    service=chat_completion_o4_minit,
    # Enable function calling if needed
)


thread: ChatHistoryAgentThread = ChatHistoryAgentThread()
response = await agent.get_response(messages="Solve these equations: 2x + 5y = 14, 3x - 4y = 5. First think about how to solve this. YOU MUST GIVE THE FINAL ANSWER ONLY AND NO OTHER TEXT.", thread=thread)

async for message in thread.get_messages():  # Get all messages in the thread
    print(message.to_dict())  # Print each message as a dictionary

thread._chat_history.model_dump()["messages"]  # Get the chat history as a dictionary
# await thread.delete() if thread else None


{'role': 'user', 'content': 'Solve these equations: 2x + 5y = 14, 3x - 4y = 5. First think about how to solve this. YOU MUST GIVE THE FINAL ANSWER ONLY AND NO OTHER TEXT.'}
{'role': 'assistant', 'content': 'x = 81/23, y = 32/23', 'name': 'ReasoningAgent'}


[{'ai_model_id': None,
  'metadata': {'agent_id': '44aabc37-09af-473e-bde6-3cc5f2e8cd0f'},
  'content_type': 'message',
  'role': <AuthorRole.USER: 'user'>,
  'name': None,
  'items': [{'ai_model_id': None,
    'metadata': {},
    'content_type': 'text',
    'text': 'Solve these equations: 2x + 5y = 14, 3x - 4y = 5. First think about how to solve this. YOU MUST GIVE THE FINAL ANSWER ONLY AND NO OTHER TEXT.',
    'encoding': None}],
  'encoding': None,
  'finish_reason': None,
  'status': None},
 {'ai_model_id': 'o4-mini',
  'metadata': {'logprobs': None,
   'id': 'chatcmpl-BuJVxokPRahl1ku0l15mFfY5mbORH',
   'created': 1752761081,
   'system_fingerprint': None,
   'usage': {'prompt_tokens': 70,
    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0},
    'completion_tokens': 480,
    'completion_tokens_details': {'accepted_prediction_tokens': 0,
     'audio_tokens': 0,
     'reasoning_tokens': 448,
     'rejected_prediction_tokens': 0}}},
  'content_type': 'message',
  'rol

In [8]:
from semantic_kernel.contents import ChatMessageContent

from semantic_kernel.agents import GroupChatOrchestration, RoundRobinGroupChatManager, GroupChatManager


In [ ]:
## Group chat orchestration

def get_agents() -> list[Agent]:
    writer = ChatCompletionAgent(
        name="Writer",
        description="A content writer.",
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
        service=chat_completion,
    )
    reviewer = ChatCompletionAgent(
        name="Reviewer",
        description="A content reviewer.",
        instructions=(
            "You are an excellent content reviewer. You review the content and provide feedback to the writer."
        ),
        service=chat_completion,
    )
    return [writer, reviewer]


def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"**{message.name}**\n{message.content}")
    

agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    manager=RoundRobinGroupChatManager(max_rounds=5),  # Odd number so writer gets the last word
    agent_response_callback=agent_response_callback,
)

from semantic_kernel.agents.runtime import InProcessRuntime

runtime = InProcessRuntime()
runtime.start()

**Writer**
Electrify the journey—fun to drive, easy on your wallet.
**Reviewer**
Here’s some feedback on your slogan “Electrify the journey—fun to drive, easy on your wallet.”  

What’s working  
• Clear benefit messaging: “fun to drive” and “easy on your wallet” immediately communicate the SUV’s two key selling points.  
• Active language: “Electrify the journey” has a nice verb that ties back to the electric theme.  
• Straightforward and honest: it won’t overpromise or confuse the buyer.  

Opportunities to tighten and elevate  
1) Make it more concise and punchy. Right now it reads like two clauses joined by a dash—consider a shorter, single‐idea hook.  
2) Add emotional or aspirational flavor. “Journey” is a solid start, but can you evoke freedom, adventure or style?  
3) Play with rhythm or wordplay. A little alliteration or rhyme can make it stick in the mind.  

Possible refinements  
• “Charge into Fun. Drive into Savings.”  
• “Electrify Adventure, Not Your Expenses.”  
• “Pl

In [68]:
orchestration_result = await group_chat_orchestration.invoke(
    task="Create a slogan for a new electric SUV that is affordable and fun to drive.",
    runtime=runtime,
)

In [69]:
value = await orchestration_result.get()
print(f"***** Final Result *****\n{value}")

await runtime.stop_when_idle()

***** Final Result *****
Charge Up Your Adventure. Save Every Mile.


In [4]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.functions import KernelArguments
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from semantic_kernel.contents import AuthorRole, ChatHistory, ChatMessageContent
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from typing_extensions import override

In [80]:
## Custom LLM based group chat manager
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the group style discussion.

    Feel free to add or remove agents.
    """
    farmer = ChatCompletionAgent(
        name="Farmer",
        description="A rural farmer from Southeast Asia.",
        instructions=(
            "You're a farmer from Southeast Asia. "
            "Your life is deeply connected to land and family. "
            "You value tradition and sustainability. "
            "You are in a debate. Feel free to challenge the other participants with respect."
        ),
        service=chat_completion,
    )
    developer = ChatCompletionAgent(
        name="Developer",
        description="An urban software developer from the United States.",
        instructions=(
            "You're a software developer from the United States. "
            "Your life is fast-paced and technology-driven. "
            "You value innovation, freedom, and work-life balance. "
            "You are in a debate. Feel free to challenge the other participants with respect."
        ),
        service=chat_completion,
    )
    
    return [farmer, developer]

class ChatCompletionGroupChatManager(GroupChatManager):
    """A simple chat completion base group chat manager.

    This chat completion service requires a model that supports structured output.
    """

    service: ChatCompletionClientBase

    topic: str

    termination_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to determine if the discussion has reached a conclusion. "
        "If you would like to end the discussion, please respond with True. Otherwise, respond with False."
    )

    selection_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to select the next participant to speak. "
        "Here are the names and descriptions of the participants: "
        "{{$participants}}\n"
        "Please respond with only the name of the participant you would like to select."
    )

    result_filter_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You have just concluded the discussion. "
        "Please summarize the discussion and provide a closing statement."
    )

    def __init__(self, topic: str, service: ChatCompletionClientBase, **kwargs) -> None:
        """Initialize the group chat manager."""
        super().__init__(topic=topic, service=service, **kwargs)

    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Helper to render a prompt with arguments."""
        prompt_template_config = PromptTemplateConfig(template=prompt)
        prompt_template = KernelPromptTemplate(prompt_template_config=prompt_template_config)
        return await prompt_template.render(Kernel(), arguments=arguments)

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Provide concrete implementation for determining if user input is needed.

        The manager will check if input from human is needed after each agent message.
        """
        return BooleanResult(
            result=False,
            reason="This group chat manager does not require user input.",
        )

    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """Provide concrete implementation for determining if the discussion should end.

        The manager will check if the conversation should be terminated after each agent message
        or human input (if applicable).
        """
        should_terminate = await super().should_terminate(chat_history)
        if should_terminate.result:
            return should_terminate

        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.termination_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Determine if the discussion should end."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=BooleanResult),
        )

        termination_with_reason = BooleanResult.model_validate_json(response.content)

        print("*********************")
        print(f"Should terminate: {termination_with_reason.result}\nReason: {termination_with_reason.reason}.")
        print("*********************")

        return termination_with_reason

    @override
    async def select_next_agent(
        self,
        chat_history: ChatHistory,
        participant_descriptions: dict[str, str],
    ) -> StringResult:
        """Provide concrete implementation for selecting the next agent to speak.

        The manager will select the next agent to speak after each agent message
        or human input (if applicable) if the conversation is not terminated.
        """
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.selection_prompt,
                    KernelArguments(
                        topic=self.topic,
                        participants="\n".join([f"{k}: {v}" for k, v in participant_descriptions.items()]),
                    ),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Now select the next participant to speak."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )

        participant_name_with_reason = StringResult.model_validate_json(response.content)

        print("*********************")
        print(
            f"Next participant: {participant_name_with_reason.result}\nReason: {participant_name_with_reason.reason}."
        )
        print("*********************")

        if participant_name_with_reason.result in participant_descriptions:
            return participant_name_with_reason

        raise RuntimeError(f"Unknown participant selected: {response.content}.")

    @override
    async def filter_results(
        self,
        chat_history: ChatHistory,
    ) -> MessageResult:
        """Provide concrete implementation for filtering the results of the discussion.

        The manager will filter the results of the conversation after the conversation is terminated.
        """
        if not chat_history.messages:
            raise RuntimeError("No messages in the chat history.")

        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.result_filter_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Please summarize the discussion."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )
        string_with_reason = StringResult.model_validate_json(response.content)

        return MessageResult(
            result=ChatMessageContent(role=AuthorRole.ASSISTANT, content=string_with_reason.result),
            reason=string_with_reason.reason,
        )
        
def agent_response_callback(message: ChatMessageContent) -> None:
    """Callback function to retrieve agent responses."""
    print(f"**{message.name}**\n{message.content}")
   
agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    manager=ChatCompletionGroupChatManager(
        topic="What does a good life mean to you personally?",
        service=chat_completion,
        max_rounds=10,
    ),
    agent_response_callback=agent_response_callback,
)

runtime = InProcessRuntime()
runtime.start()

orchestration_result = await group_chat_orchestration.invoke(
    task="Please start the discussion.",
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get()
print(value)

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()



*********************
Should terminate: False
Reason: Discussion has just started and no conclusion reached..
*********************
*********************
Next participant: Farmer
Reason: Starting with the Farmer provides a perspective from rural life, offering a foundational contrast to urban views..
*********************
**Farmer**
Sawatdee khráp, everyone. My name is Somchai, and I’ve been tending my family’s rice terraces in northern Thailand for more than twenty years. Today I’d like to open a respectful debate on how best to feed our communities while caring for the land that sustains us.

Opening Points:
1. Tradition and Sustainability  
   • For generations, we’ve used crop rotation, green manure and water-saving techniques passed down by our elders.  
   • These methods build soil fertility naturally, protect local biodiversity and keep input costs low.  

2. Modern Intensification  
   • Others argue that chemical fertilizers, heavy irrigation and high-yield seed varieties are

In [9]:
## Structured response
import json
from pydantic import BaseModel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings


In [ ]:

# Define the BaseModel we will use for structured outputs
class Step(BaseModel):
    explanation: str
    output: str


class Reasoning(BaseModel):
    steps: list[Step]
    final_answer: str
    
USER_INPUT = "how can I solve 8x + 7y = -23, and 4x=12?"

settings = OpenAIChatPromptExecutionSettings() # AzureChatPromptExecutionSettings()
settings.response_format = Reasoning

agent = ChatCompletionAgent(
        service=chat_completion,
        name="Assistant",
        instructions="Answer the user's questions.",
        arguments=KernelArguments(settings=settings)
    )

thread: ChatHistoryAgentThread = None

print(f"# User: {USER_INPUT}")
# 4. Invoke the agent for a response
response = await agent.get_response(messages=USER_INPUT, thread=thread)
# 5. Validate the response and print the structured output
reasoned_result = Reasoning.model_validate(json.loads(response.message.content))
print(f"# {response.name}:\n\n{reasoned_result.model_dump_json(indent=4)}")

# 6. Cleanup: Clear the thread
await thread.delete() if thread else None


# User: how can I solve 8x + 7y = -23, and 4x=12?
# Assistant:

{
    "steps": [
        {
            "explanation": "Solve the simple equation 4x = 12 for x.",
            "output": "x = 12/4 = 3"
        },
        {
            "explanation": "Substitute x = 3 into 8x + 7y = -23 and solve for y.",
            "output": "8·3 + 7y = -23 ⇒ 24 + 7y = -23 ⇒ 7y = -47 ⇒ y = -47/7"
        }
    ],
    "final_answer": "The solution is x = 3 and y = -47/7."
}


In [23]:
## Section below for multi-agent orchestration

In [10]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, ConcurrentOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

In [ ]:

def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the concurrent orchestration.

    Feel free to add or remove agents.
    """
    physics_agent = ChatCompletionAgent(
        name="PhysicsExpert",
        instructions="You are an expert in physics. You answer questions from a physics perspective.",
        service=chat_completion,
    )
    chemistry_agent = ChatCompletionAgent(
        name="ChemistryExpert",
        instructions="You are an expert in chemistry. You answer questions from a chemistry perspective.",
        service=chat_completion,
    )

    return [physics_agent, chemistry_agent]

agents = get_agents()
concurrent_orchestration = ConcurrentOrchestration(members=agents)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await concurrent_orchestration.invoke(
    task="What is temperature?",
    runtime=runtime,
)

# 4. Wait for the results
# Note: the order of the results is not guaranteed to be the same
# as the order of the agents in the orchestration.
value = await orchestration_result.get(timeout=60)
for item in value:
    print(f"# {item.name}: {item.content}")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()

# ChemistryExpert: Temperature is a basic physical property that expresses how “hot” or “cold” a system is.  From a chemistry (and physics) standpoint you can look at it in three complementary ways:

1. Microscopic (kinetic) view  
   •  In a gas, liquid or solid the constituent atoms or molecules are in constant motion—vibrating, rotating, translating.  
   •  Temperature is proportional to the average kinetic energy of those particles.  Higher temperature means, on average, faster motion.  

2. Thermodynamic (macroscopic) view  
   •  Temperature is an intensive property: it does not depend on the amount of material.  
   •  In thermodynamics one defines it via the relation  
        dU = T dS − P dV  
     (U = internal energy, S = entropy, P = pressure, V = volume).  Here T is the thermodynamic temperature, which ensures that if two systems at different T come into contact, heat flows from higher to lower T until equilibrium.  

3. Practical measurement and units  
   •  The SI uni

In [11]:
from semantic_kernel.agents.orchestration.tools import structured_outputs_transform

In [44]:
# With structured output
class Definition(BaseModel):
    """A model to hold the analysis of an article."""
    definition: str
    unit: str

class Perspectives(BaseModel):
    """A model to hold the perspectives of different agents."""
    physics: Definition
    chemistry: Definition

agents = get_agents()
concurrent_orchestration = ConcurrentOrchestration[str, Perspectives](
    members=agents,
    output_transform=structured_outputs_transform(Perspectives, chat_completion),
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await concurrent_orchestration.invoke(
    task="Topic: Temperature. Give a definition and unit.",
    runtime=runtime,
)

value = await orchestration_result.get(timeout=20)
if isinstance(value, Perspectives):
    print(value.model_dump_json(indent=2))
else:
    print("Unexpected result type:", type(value))

# 6. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()

{
  "physics": {
    "definition": "Temperature is an intensive thermodynamic state variable that determines the direction of heat flow between bodies (from higher to lower temperature). Microscopically, it is proportional to the average kinetic energy per degree of freedom of particles: ⟨Ekin⟩ = (f/2) kB T, where f is degrees of freedom and kB is the Boltzmann constant.",
    "unit": "Kelvin (K)"
  },
  "chemistry": {
    "definition": "Temperature is a measure of the average kinetic energy of the particles in a substance. It dictates heat flow direction—spontaneously from higher to lower temperature—until thermal equilibrium is reached.",
    "unit": "Kelvin (K)"
  }
}


In [55]:
## Sequeential orchestration
from semantic_kernel.agents import Agent, ChatCompletionAgent, SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime


def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the sequential orchestration.

    Feel free to add or remove agents.
    """
    concept_extractor_agent = ChatCompletionAgent(
        name="ConceptExtractorAgent",
        instructions=(
            "You are a marketing analyst. Given a product description, identify:\n"
            "- Key features\n"
            "- Target audience\n"
            "- Unique selling points\n\n"
        ),
        service=chat_completion,
    )
    writer_agent = ChatCompletionAgent(
        name="WriterAgent",
        instructions=(
            "You are a marketing copywriter. Given a block of text describing features, audience, and USPs, "
            "compose a compelling marketing copy (like a newsletter section) that highlights these points. "
            "Output should be short (around 150 words), output just the copy as a single text block."
        ),
        service=chat_completion,
    )
    format_proof_agent = ChatCompletionAgent(
        name="FormatProofAgent",
        instructions=(
            "You are an editor. Given the draft copy, correct grammar, improve clarity, ensure consistent tone, "
            "give format and make it polished. Output the final improved copy as a single text block."
        ),
        service=chat_completion,
    )

    # The order of the agents in the list will be the order in which they are executed
    return [concept_extractor_agent, writer_agent, format_proof_agent]

def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"# {message.name}\n{message.content}")


agents = get_agents()
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await sequential_orchestration.invoke(
    task="An eco-friendly stainless steel water bottle that keeps drinks cold for 24 hours",
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get(timeout=20)
print(f"***** Final Result *****\n{value}")

# 5. Stop the runtime when idle
await runtime.stop_when_idle()


# ConceptExtractorAgent
Key Features  
• Double-wall vacuum-insulated stainless steel construction  
• Keeps beverages cold for up to 24 hours  
• Eco-friendly and BPA-free materials  
• Leak-proof, sweat-free exterior  
• Available in multiple sizes and finishes  

Target Audience  
• Environmentally conscious consumers looking to reduce single-use plastic  
• Outdoor enthusiasts (hikers, campers, bikers)  
• Fitness-savvy individuals and gym-goers  
• Commuters and office workers seeking long-lasting hydration  
• Students and travelers in need of reliable drinkware  

Unique Selling Points  
• Best-in-class 24-hour cold retention—outperforms most competitors  
• Sustainable, non-toxic stainless steel promotes a zero-waste lifestyle  
• Durable design that resists dents, scratches and odors  
• Sweat-proof exterior keeps hands and bags dry  
• Versatile style options to suit personal taste and brand partnerships  
# WriterAgent
Meet AquaFlow, the last water bottle you’ll ever need. C

In [58]:
from semantic_kernel.contents import StreamingChatMessageContent
is_new_message = True

## Introduce streaming agent response callback

def streaming_agent_response_callback(message: StreamingChatMessageContent, is_final: bool) -> None:
    """Observer function to print the messages from the agents.

    Args:
        message (StreamingChatMessageContent): The streaming message content from the agent.
        is_final (bool): Indicates if this is the final part of the message.
    """
    global is_new_message
    if is_new_message:
        print(f"# {message.name}")
        is_new_message = False
    print(message.content, end="", flush=True)
    if is_final:
        print()
        is_new_message = True
        
agents = get_agents()
sequential_orchestration = SequentialOrchestration(
    members=agents,
    streaming_agent_response_callback=streaming_agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await sequential_orchestration.invoke(
    task="An eco-friendly stainless steel water bottle that keeps drinks cold for 24 hours",
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get(timeout=60)
print(f"***** Final Result *****\n{value}")

# 5. Stop the runtime when idle
await runtime.stop_when_idle()


# ConceptExtractorAgent
Key Features  
• Premium stainless steel construction for durability and hygiene  
• Double-wall vacuum insulation keeps beverages cold up to 24 hours  
• Eco-friendly, BPA-free and fully recyclable materials  
• Leak-proof, screw-top lid with a food-grade silicone seal  
• Condensation-free exterior to keep hands and bags dry  

Target Audience  
• Environmentally conscious consumers looking to reduce single-use plastics  
• Outdoor enthusiasts (hikers, campers, cyclists) needing reliable hydration  
• Fitness buffs and athletes seeking long-lasting cold drinks during workouts  
• Commuters, students, and office workers who want a stylish, reusable bottle  
• Travelers who need a durable vessel for all-day refreshment  

Unique Selling Points  
• Industry-leading 24-hour cold retention—outperforms standard insulated bottles  
• A truly sustainable alternative to disposable plastic bottles  
• High-grade stainless steel resists dents, rust, and flavor transfer  

In [12]:
from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatHistory, ChatMessageContent


In [63]:
## Group chat with human in the loop
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the group style discussion.

    Feel free to add or remove agents.
    """
    writer = ChatCompletionAgent(
        name="Writer",
        description="A content writer.",
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
        service=chat_completion,
    )
    reviewer = ChatCompletionAgent(
        name="Reviewer",
        description="A content reviewer.",
        instructions=(
            "You are an excellent content reviewer. You review the content and provide feedback to the writer."
        ),
        service=chat_completion,
    )

    # The order of the agents in the list will be the order in which they will be picked by the round robin manager
    return [writer, reviewer]


class CustomRoundRobinGroupChatManager(RoundRobinGroupChatManager):
    """Custom round robin group chat manager to enable user input."""

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Override the default behavior to request user input after the reviewer's message.

        The manager will check if input from human is needed after each agent message.
        """
        if len(chat_history.messages) == 0:
            return BooleanResult(
                result=False,
                reason="No agents have spoken yet.",
            )
        last_message = chat_history.messages[-1]
        if last_message.name == "Reviewer":
            return BooleanResult(
                result=True,
                reason="User input is needed after the reviewer's message.",
            )

        return BooleanResult(
            result=False,
            reason="User input is not needed if the last message is not from the reviewer.",
        )


def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"**{message.name}**\n{message.content}")


async def human_response_function(chat_histoy: ChatHistory) -> ChatMessageContent:
    """Function to get user input."""
    user_input = input("User: ")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    # max_rounds is odd, so that the writer gets the last round
    manager=CustomRoundRobinGroupChatManager(
        max_rounds=5,
        human_response_function=human_response_function,
    ),
    agent_response_callback=agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await group_chat_orchestration.invoke(
    task="Create a slogan for a new electric SUV that is affordable and fun to drive.",
    runtime=runtime,
)

# 4. Wait for the results
value = await orchestration_result.get()
print(f"***** Result *****\n{value}")


**Writer**
“Charge into Fun—Affordable Electric Adventure”
**Reviewer**
Here’s some feedback on your slogan “Charge into Fun—Affordable Electric Adventure”:

Strengths  
• “Charge” cleverly evokes electric power.  
• “Fun” communicates the driving experience.  
• You’ve covered the key selling points: electric, affordable, adventure.

Areas to refine  
• Rhythm & brevity: The phrase “Affordable Electric Adventure” feels a bit long and abstract.  
• Emotional pull: “Adventure” is great, but pairing it more tightly with everyday practicality could boost appeal.  
• Word choice: Consider swapping “electric” for a more evocative synonym (e.g. “spark,” “power,” “juice”) or tightening the phrase.

Possible revisions  
1. “Spark Your Adventure—Electric Fun, Wallet-Friendly”  
2. “Electrify the Road, Not Your Budget”  
3. “Affordable Spark. Unforgettable Drive.”  
4. “Power Up the Fun—Adventure Within Reach”  
5. “Drive Electric, Drive Happy, Spend Less”

Any of these could be further tweaked 

Task was destroyed but it is pending!
task: <Task pending name='Task-75' coro=<RunContext._run() running at /home/agangwal/lseg-migration-agent/.venv/lib/python3.12/site-packages/semantic_kernel/agents/runtime/in_process/in_process_runtime.py:124> wait_for=<Future pending cb=[Task.task_wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-88' coro=<RunContext._run() running at /home/agangwal/lseg-migration-agent/.venv/lib/python3.12/site-packages/semantic_kernel/agents/runtime/in_process/in_process_runtime.py:124> wait_for=<Future pending cb=[Task.task_wakeup()]>>


***** Result *****
It looks like you didn’t specify—would you like to pick one of those rhymes, see more options, or refine a particular line further? Let me know how you’d like to proceed.


In [ ]:
## This is where we can move the group chat example from further up that shows an llm based group chat manager to here.

In [6]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.agents import Agent, ChatCompletionAgent, HandoffOrchestration, OrchestrationHandoffs
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import kernel_function


from semantic_kernel.functions import kernel_function


class OrderStatusPlugin:
    @kernel_function
    def check_order_status(self, order_id: str) -> str:
        """Check the status of an order."""
        # Simulate checking the order status
        return f"Order {order_id} is shipped and will arrive in 2-3 days."


class OrderRefundPlugin:
    @kernel_function
    def process_refund(self, order_id: str, reason: str) -> str:
        """Process a refund for an order."""
        # Simulate processing a refund
        print(f"Processing refund for order {order_id} due to: {reason}")
        return f"Refund for order {order_id} has been processed successfully."


class OrderReturnPlugin:
    @kernel_function
    def process_return(self, order_id: str, reason: str) -> str:
        """Process a return for an order."""
        # Simulate processing a return
        print(f"Processing return for order {order_id} due to: {reason}")
        return f"Return for order {order_id} has been processed successfully."

support_agent = ChatCompletionAgent(
    name="TriageAgent",
    description="A customer support agent that triages issues.",
    instructions="Handle customer requests. ONLY HAND OFF TO USER IF YOU NEED MORE INFORMATION.",
    service=chat_completion,
)

refund_agent = ChatCompletionAgent(
    name="RefundAgent",
    description="A customer support agent that handles refunds.",
    instructions="Handle refund requests. ONLY HAND OFF TO USER IF YOU NEED MORE INFORMATION.",
    service=chat_completion,
    plugins=[OrderRefundPlugin()],
)

order_status_agent = ChatCompletionAgent(
    name="OrderStatusAgent",
    description="A customer support agent that checks order status.",
    instructions="Handle order status requests. ONLY HAND OFF TO USER IF YOU NEED MORE INFORMATION.",
    service=chat_completion,
    plugins=[OrderStatusPlugin()],
)

order_return_agent = ChatCompletionAgent(
    name="OrderReturnAgent",
    description="A customer support agent that handles order returns.",
    instructions="Handle order return requests. ONLY HAND OFF TO USER IF YOU NEED MORE INFORMATION.",
    service=chat_completion,
    plugins=[OrderReturnPlugin()],
)

from semantic_kernel.agents import OrchestrationHandoffs

handoffs = (
    OrchestrationHandoffs()
    .add_many(    # Use add_many to add multiple handoffs to the same source agent at once
        source_agent=support_agent.name,
        target_agents={
            refund_agent.name: "Transfer to this agent if the issue is refund related",
            order_status_agent.name: "Transfer to this agent if the issue is order status related",
            order_return_agent.name: "Transfer to this agent if the issue is order return related",
        },
    )
    .add(    # Use add to add a single handoff
        source_agent=refund_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not refund related",
    )
    .add(
        source_agent=order_status_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not order status related",
    )
    .add(
        source_agent=order_return_agent.name,
        target_agent=support_agent.name,
        description="Transfer to this agent if the issue is not order return related",
    )
)


def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents.

    Please note that this function is called whenever the agent generates a response,
    including the internal processing messages (such as tool calls) that are not visible
    to other agents in the orchestration.
    """
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Calling '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"Result from '{item.name}' is '{item.result}'")
    

from semantic_kernel.agents import HandoffOrchestration
def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

handoff_orchestration = HandoffOrchestration(
    members=[
        support_agent,
        refund_agent,
        order_status_agent,
        order_return_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=agent_response_callback,
    human_response_function=human_response_function,
)

runtime = InProcessRuntime()
runtime.start()


orchestration_result = await handoff_orchestration.invoke(
    task="A customer is on the line. Ask them about their issue.",
    runtime=runtime,
)

value = await orchestration_result.get()
print(value)


await runtime.stop_when_idle()  # Stop the runtime when idle

TriageAgent: Thank you for reaching out. Could you please let me know what issue you’re experiencing with your order or service? I’m here to help!
TriageAgent: 
Calling 'Handoff-transfer_to_OrderStatusAgent' with arguments '{}'
TriageAgent: 
Result from 'Handoff-transfer_to_OrderStatusAgent' is 'None'
TriageAgent: 
OrderStatusAgent: Could you please provide your order number? I’ll check the status for you right away.
OrderStatusAgent: 
Calling 'OrderStatusPlugin-check_order_status' with arguments '{"order_id":"32422"}'
OrderStatusAgent: 
Result from 'OrderStatusPlugin-check_order_status' is 'Order 32422 is shipped and will arrive in 2-3 days.'
OrderStatusAgent: Your order (32422) has been shipped and is expected to arrive in 2–3 days. If you need more details or have another question, please let me know!
Task is completed with summary: The customer inquired about the status of order 32422. I confirmed the order has shipped and will arrive in 2-3 days. The customer confirmed no further 